# Theme 3: Deep Learning Hands-On Walkthrough

This notebook allows us to step through the logic of `solve_theme3.py` interactively.
We will explore:
1. Data Loading & Preprocessing
2. Building a simple Linear Classifier (Single Layer Neural Network)
3. The effects of Training Duration (Epochs)
4. Visualizing Neural Network Weights
5. Regularization (L2)
6. Deepening the Network (Adding Hidden Layers)
7. Activation Functions

## Self-Learning Goal
The goal here is not just to run code, but to understand *why* each step is necessary and what the output tells us about the model's learning process.


## 1. Imports

We use `numpy` for data manipulation, `tensorflow/keras` for building neural networks, `PIL` for image handling, and `matplotlib` for visualization.


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Flatten
from keras.models import Model
from keras import regularizers
from PIL import Image
import matplotlib.pyplot as plt
import os

print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.20.0


c:\Users\danie\Documents\Projects\ml4cv\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


## 2. Helper Functions

In Deep Learning, data plumbing is often 80% of the work. We need robust functions to:
- **Load Labels**: Read class IDs from text files.
- **Load Images**: Read image files into NumPy arrays.
- **Normalize**: Scale pixel values to a small range (e.g., -1 to 1 or 0 to 1) to help the optimizer converge.
- **Split**: Separate data into Testing and Validation sets to ensure we don't cheat by optimizing on our test data.

*Self-Learning Note*: Notice `normalize_dataset`. It shifts data to be 0-centered (approx) by subtracting 128 and dividing by 128. This helps gradient descent steps be more uniform.


In [1]:
# --- Helper Functions ---
def load_labels(filename):
    with open(filename,'r') as file:
       li = file.readlines()
    label_count = len(li)
    labels = np.empty((label_count,1), dtype='int')
    i = 0
    with open(filename) as f:
        for line in f:
            labels[i] = int(line.replace("\n", ""))
            i = i + 1
    return labels

def load_images(folder, image_count, image_size):
    array_shape = (image_count, image_size[0], image_size[1], image_size[2])
    imageset = np.empty(array_shape, dtype='float')
    for i in range(0,image_count):
        fname = os.path.join(folder, 'image_' + "{:04d}".format(i) + '.png')
        if not os.path.exists(fname):
             print(f"Warning: File {fname} not found.")
             continue
        image = Image.open(fname)
        imageset[i] = np.asarray(image)
    return imageset

def normalize_dataset(sampled_images):
    # Scale to range [-1, 1] approx
    sampled_images = (sampled_images.astype('float32')-128) / 128
    return sampled_images

def split_test_val(data, splitpoint):
    # Returns (Tail, Head) - careful with indices!
    return data[splitpoint:], data[:splitpoint]

def create_model(input_shape, dense_size, classes, l2_reg=None):
    x = Input(shape=(input_shape))
    y = Flatten()(x)
    
    if l2_reg:
        y = Dense(classes, activation='softmax', name='dense_layer', kernel_regularizer=l2_reg)(y)
    else:
        y = Dense(classes, activation='softmax', name='dense_layer')(y)
        
    model = Model(inputs=x, outputs=y)
    return model

def create_model_2_layers(input_shape, dense_size, classes):
    x = Input(shape=(input_shape))
    y = Flatten()(x)
    # Task 7: 2nd dense layer usually adds 'depth' allowing complex features
    y = Dense(dense_size, activation='relu')(y)
    y = Dense(classes, activation='softmax', name='dense_layer_out')(y)
    model = Model(inputs=x, outputs=y)
    return model

def visualize_weights(model, layer_name, title, filename):
    try:
        weights, biases = model.get_layer(layer_name).get_weights()
    except ValueError:
        print(f"Layer {layer_name} not found or has no weights")
        return

    # Weights shape: (InputDim, Units) -> (3072, 10)
    input_shape = (32, 32, 3)
    n_units = weights.shape[1]
    
    # We want to show the weights for each class (unit)
    rows = 2
    cols = 5 
    
    fig, axes = plt.subplots(rows, cols, figsize=(10, 5))
    
    for i, ax in enumerate(axes.flat):
        if i >= n_units:
            break

        weight_vector = weights[:, i]
        # Reshape: (3072,) -> (32, 32, 3)
        weight_image = weight_vector.reshape(input_shape)

        # Normalize to 0-1 for display
        min_val = weight_image.min()
        max_val = weight_image.max()
        weight_image = (weight_image - min_val) / (max_val - min_val + 1e-5)

        ax.imshow(weight_image)
        ax.set_title(f'Class {i}')
        ax.axis('off')

    plt.suptitle(title)
    plt.tight_layout()
    plt.show() # Changed from savefig to show for notebook
    print(f"Visualization generated.")


## 3. Loading Data

We load the CIFAR-10 subset.
We assume the `resources` folder is in the current working directory.


In [ ]:
print("--- Loading Data ---")
base_res = 'resources' # Assumed relative to CWD
if not os.path.exists(base_res):
    print(f"ERROR: '{base_res}' directory not found. Please ensure you are in the correct directory.")
else:
    # 1. Load Labels
    y_train = load_labels(os.path.join(base_res, 'training/labels.csv'))
    
    # 2. Load Images (based on how many labels we have)
    x_train_raw = load_images(os.path.join(base_res, 'training'), len(y_train), (32,32,3))
    
    Y_test = load_labels(os.path.join(base_res, 'testing/labels.csv'))
    X_test_raw = load_images(os.path.join(base_res, 'testing'), len(Y_test), (32,32,3))

    print("Data Loaded.")

## 4. Normalization & Splitting

**Normalization**: Raw pixel values are 0-255. Neural networks prefer inputs near 0 with unit variance. We subtract 128 and divide by 128.

**Splitting**: We split the 'Test' set folder into a true 'Test' set and a 'Validation' set.
- **Validation**: Used during training to check progress.
- **Test**: Used ONLY at the very end to report final performance.


In [ ]:
x_train = normalize_dataset(x_train_raw)
X_test = normalize_dataset(X_test_raw)

# Splitting logic from original script
# index 0 to 2000 -> Validation
# index 2000 to End -> Test
splitpoint = 2000
x_test, x_val = split_test_val(X_test, splitpoint)
y_test, y_val = split_test_val(Y_test, splitpoint)

class_count = len(np.unique(y_train))
dims = (32, 32, 3)
dense_sz = 100

print(f"Training set: {x_train.shape}")
print(f"Validation set: {x_val.shape}")
print(f"Test set: {x_test.shape}")
print(f"Classes: {class_count}")

## 5. Task 3: Baseline Model (5 Epochs)

We create a simple model: `Input -> Flatten -> Dense (Softmax)`.
This is effectively Multinomial Logistic Regression.
It has no "hidden" layers, so it can only learn linear boundaries between classes.

We train for just 5 epochs to sanity check that learning happens.


In [ ]:
print("\n--- Task 3 (Standard Model 5 epochs) ---")
model = create_model(dims, dense_sz, class_count)
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5), metrics=['accuracy'])
model.summary()

l1 = model.get_layer('dense_layer')
w, b = l1.get_weights()
print(f"Linear Layer Weights shape: {w.shape}")

model.fit(x_train, y_train, epochs=5, validation_data=(x_val, y_val), verbose=2)
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test accuracy (5 epochs): {score[1]}')

## 6. Task 4: Extended Training (100 Epochs)

5 Epochs is rarely enough. Let's train for 100 epochs and plot the history.

**Self-Learning Observation**:
- If `train_acc` keeps going up but `val_acc` plateaus or goes down, we are **overfitting**.
- If both are low, we are **underfitting**.


In [ ]:
print("\n--- Task 4 (Standard Model 100 epochs) ---")
model_t4 = create_model(dims, dense_sz, class_count)
model_t4.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5), metrics=['accuracy'])
history = model_t4.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val), verbose=0)

plt.figure()
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.title('Task 4: Accuracy vs Epochs')
plt.legend()
plt.show()

print(f"Final Train Acc: {history.history['accuracy'][-1]:.4f}")
print(f"Final Val Acc: {history.history['val_accuracy'][-1]:.4f}")

## 7. Task 5: Visualizing Weights

Since we have a linear model (no hidden layers), the weights connecting the input pixels to a class can be interpreted as a "feature template".
If the model thinks a "Car" looks like a blob with wheels, the weights for the "Car" class will look like a ghost image of a car.

**Observation**: High noise in these images suggests the model is memorizing statisical noise rather than learning robust features.


In [ ]:
print("\n--- Task 5 (Visualization) ---")
visualize_weights(model_t4, 'dense_layer', 'Task 5: Weights', 'task5_weights.png')

## 8. Task 6: L2 Regularization

To reduce noise and overfitting, we apply **L2 Regularization**.
This forces weights to be small. Large weights usually mean the model is trying too hard to fit specific noisy pixels. Small weights imply a smoother, more general solution.

**Expectation**:
- Weights should look "smoother" or "blurrier" in visualization.
- Validation accuracy might improve (or at least the gap between train/val should close).


In [ ]:
print("\n--- Task 6 (L2 Regularization) ---")
model_l2 = create_model(dims, dense_sz, class_count, l2_reg=regularizers.L2(0.03))
model_l2.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5), metrics=['accuracy'])
history_l2 = model_l2.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val), verbose=0)

# Visualize new weights
visualize_weights(model_l2, 'dense_layer', 'Task 6: L2 Weights', 'task6_weights.png')

plt.figure()
plt.plot(history_l2.history['accuracy'], label='train_accuracy')
plt.plot(history_l2.history['val_accuracy'], label='val_accuracy')
plt.title('Task 6: L2 Accuracy')
plt.legend()
plt.show()

score_l2 = model_l2.evaluate(x_test, y_test, verbose=0)
print(f'Test accuracy (L2, 100 epochs): {score_l2[1]}')

## 9. Task 7: Two-Layer Network

Linear models can't solve XOR problems or understand hierarchical features.
We add a helper layer (Hidden Layer) with `ReLU` activation.
`Input -> Flatten -> Dense(100, ReLU) -> Dense(10, Softmax)`

This usually boosts accuracy significantly (e.g., +10%).


In [ ]:
print("\n--- Task 7 (2 Layers) ---")
model_2l = create_model_2_layers(dims, dense_sz, class_count)
model_2l.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5), metrics=['accuracy'])
history_2l = model_2l.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val), verbose=0)
score_2l = model_2l.evaluate(x_test, y_test, verbose=0)
print(f'Test accuracy (2 Layers, 100 epochs): {score_2l[1]}')

## 10. Task 8: Activation Functions Demo

Different activations have different properties:
- **Sigmoid**: Squisies output to [0, 1]. Can cause vanishing gradients.
- **ReLU**: `max(0, x)`. Fast, solves vanishing gradient for positive values.
- **LeakyReLU**: allows small negative gradient to flow.

Let's test them on a dummy matrix.


In [ ]:
print("\n--- Task 8 (Activations Demo) ---")
input_mat = np.array([[-1, 0, 1, 2], [-2, 0, 1, 2], [10, 15, 20, 30], [-20, -10, -5, -1]]).astype(np.float32)

def demo_activation(name, func):
    out = func(input_mat)
    print(f"Activation: {name}")
    print(out.numpy())

demo_activation('ReLU', keras.activations.relu)
demo_activation('Sigmoid', keras.activations.sigmoid)
l_relu = keras.layers.LeakyReLU(negative_slope=0.1)
out_lrelu = l_relu(input_mat)
print("Activation: LeakyReLU (0.1)")
print(out_lrelu.numpy())